# 3. Via Human Hematopoiesis

This notebook uses VIA to interpret the CD34 Hematopoiesis dataset (Setty et al.,2019) 

In [1]:
import pyVIA.core as via
import pyVIA.datasets_via as   datasets
import pandas as pd 
import scanpy as sc

import matplotlib.pyplot as plt
import warnings
# warnings.filterwarnings('ignore')

SyntaxError: invalid syntax (plotting_via.py, line 1569)

## Load data from directory
We use annotations given by SingleR which uses Novershtern Hematopoietic cell data as the reference. These are in line with the annotations given by Setty et al., 2019 but offer a slightly more granular annotation.

In [ ]:
adata = datasets.scRNA_hematopoiesis()
sc.tl.pca(adata, svd_solver='arpack', n_comps=200)
tsnem = adata.obsm['tsne']
adata

## User's choice of embedding
Three embedding exmaples: TSNE, UMAP, and PHATE 
Alternatively allow `via.VIA()` to compute an embedding using the underlying graph by setting: `do_embedding = True` AND `embedding_type = 'via-umap'` OR `'via-mds'`

In [ ]:
# TSNE - same as the embedding used in the original Setty et al., publication
embedding = adata.obsm['tsne']

# # UMAP
# ncomp = 30
# embedding = umap.UMAP().fit_transform(adata.obsm['X_pca'][:, 0:ncomp])

# # PHATE
# ncomp = 30
# phate_op = phate.PHATE()
# embedding = phate_op.fit_transform(adata.obsm['X_pca'][:, 0:ncomp])

## Run VIA
#### Key Parameters:
- `knn`
- `root_user` (root index of type int/ or celltype label of type `string`)
- `edgepruning_clustering_resolution` (lower number means smaller (and more) clusters) typical range 0-1
- `memory` (higher value means more focused search pathways to cell fates) typical range 2-50
- `cluster_graph_pruning` (lower number means fewer edges) typical range 0-1

NOTE: 
- If rna-velocity is available, considering using it to compute the root automatically- see RNA velocity tutorial.
- If you want to pre-specify terminal states you can do so by a list of group-level names corresponding to true-labels OR by a list of single cell indices
    - E.g. `user_defined_terminal_group=['pDC','ERY1', 'ERY3', 'MONO1','mDC (cDC)','PRE_B2']`

In [ ]:
ncomps=80
knn=30
v0_random_seed=4
root_user = [4823] #the index of a cell belonging to the HSC cell type
memory = 10

v0 = via.VIA(data=adata.obsm['X_pca'][:, 0:ncomps], true_label=adata.obs['label'], 
             edgepruning_clustering_resolution=0.15, cluster_graph_pruning=0.15,
             knn=knn,  root_user=root_user, resolution_parameter=1.5, 
             random_seed=v0_random_seed, memory=memory)#, do_compute_embedding=True, embedding_type='via-atlas')

v0.run_VIA()

## Cluster Level Trajectory Plots

In [ ]:
fig, ax, ax1 = via.plot_piechart_viagraph(via_object=v0, show_legend=False)
fig.set_size_inches(12,5)

## Visualise gene/feature graph
View the gene expression along the VIA graph. We use the computed HNSW small world graph in VIA to accelerate the gene imputation calculations (using similar approach to MAGIC) as follows:


In [ ]:
df_ = pd.DataFrame(adata.X)
df_.columns = [i for i in adata.var_names]

gene_list_magic = ['IL3RA', 'IRF8', 'GATA1', 'GATA2', 'ITGA2B', 'MPO', 'CD79B', 'SPI1', 'CD34', 'CSF1R', 'ITGAX']

#optional to do gene expression imputation

df_magic = v0.do_impute(df_, magic_steps=3, gene_list=gene_list_magic)

fig, axs = via.plot_viagraph(via_object=v0, type_data='gene', df_genes=df_magic, gene_list=gene_list_magic[0:3], arrow_head=0.1)
fig.set_size_inches(15,5)

## Trajectory projection

Visualize the overall VIA trajectory projected onto a 2D embedding (UMAP, Phate, TSNE etc) in different ways. 

1. Draw the high-level clustergraph abstraction onto the embedding;
2. Draw high-edge resolution directed graph
3. Draw a vector field/stream plot of the more fine-grained directionality of cells along the trajectory projected onto an embedding. 


#### Key Parameters:
- `scatter_size`
- `scatter_alpha`
- `linewidth`
- `draw_all_curves` (if too crowded, set to `False`)

In [ ]:
fig, ax, ax1 = via.plot_trajectory_curves(via_object=v0,embedding=tsnem, draw_all_curves=False)

In [ ]:
v0.embedding = tsnem
fig, ax = via.plot_atlas_view(via_object=v0,  n_milestones=150, sc_labels=adata.obs['label'], fontsize_labels=3, extra_title_text='Atlas View colored by pseudotime')
fig.set_size_inches(4,3)

Edge plots can be made with different edge resolutions. Run `hammerbundle_milestone_dict()` to recompute the edges for plotting. Then provide the new `hammerbundle` as a parameter to `plot_edge_bundle()`
it is better to compute the edges and save them to the `via_object`. this gives more control to the merging of edges.

In [ ]:
decay = 0.6 #increasing decay increasing merging of edges
i_bw = 0.02 #increasing bw increases merging of edges
global_visual_pruning = 0.5 #higher number retains more edges
n_milestones = 200

v0.hammerbundle_milestone_dict= via.make_edgebundle_milestone(via_object=v0, n_milestones=n_milestones, decay=decay, initial_bandwidth=i_bw, global_visual_pruning=global_visual_pruning)

In [ ]:
fig, ax = via.plot_atlas_view(via_object=v0,  add_sc_embedding=True, sc_labels_expression=adata.obs['label'], cmap='jet', sc_labels=adata.obs['label'], text_labels=True, extra_title_text='Atlas View by Cell type', fontsize_labels=3)
fig.set_size_inches(6,4)

`via_streamplot()` requires you to either provide an ndarray as embedding as an input parameter OR for via to have an embedding attribute.

In [ ]:
fig, ax = via.via_streamplot(v0, embedding=tsnem, density_grid=0.8, scatter_size=30, scatter_alpha=0.3, linewidth=0.5)
fig.set_size_inches(4,3)

In [ ]:
#Colored by pseudotime

fig, ax = via.via_streamplot(v0,density_grid=0.8, scatter_size=30, color_scheme='time', linewidth=0.5, 
                             min_mass = 1, cutoff_perc = 5, scatter_alpha=0.3, marker_edgewidth=0.1, 
                             density_stream = 2, smooth_transition=1, smooth_grid=0.5)
fig.set_size_inches(4,3)

## Probabilistic pathways and Memory
Visualize the probabilistic pathways from root to terminal state as indicated by the lineage likelihood. The higher the linage likelihood, the greater the potential of that particular cell to differentiate towards the terminal state of interest.
Changing the memory paramater will alter the specificity of the lineage pathway. 
This can be visualized at the single-cell level but also combined with the Atlas View which visualizes cell-cell connectivity and pathways

#### Key Parameters:
`marker_lineages` (`list`) of terminal clusters

In [ ]:
fig, axs= via.plot_sc_lineage_probability(via_object=v0, marker_lineages=[7,11,12,15,20,22], embedding=tsnem) #marker_lineages=v0.terminal_clusters to plot all
fig.set_size_inches(12,5)

In [ ]:
fig, axs= via.plot_atlas_view(via_object=v0, lineage_pathway=[7,11,12,15,20,22]) #marker_lineages=v0.terminal_clusters to plot all
fig.set_size_inches(12,5)

## Gene Dynamics
### Line plots
The gene dynamics along pseudotime for all detected lineages are automatically inferred by VIA. These can be interpreted as the change in gene expression along any given lineage.
#### Key Parameters
- `n_splines`
- `spline_order`
- `gene_exp` (`Dataframe`) single-cell level gene expression of select genes (gene imputation is an optional pre-step)

In [ ]:
marker_genes =['ITGA2B', 'IL3RA', 'IRF8', 'MPO', 'CSF1R', 'GATA2', 'CD79B', 'CD34']
df = pd.DataFrame(adata.X, columns = adata.var_names)
df_magic = v0.do_impute(df, magic_steps=3, gene_list=gene_list_magic) #optional
fig, axs=via.get_gene_expression(via_object=v0, gene_exp=df_magic[marker_genes])
fig.set_size_inches(14,7)

### Heatmap
Heatmaps of genes along pseudotimetime
#### Key Parameters
- `df_gene_exp` (`dataframe`) single-cell level gene expression of selected genes
- `marker_lineages` (`list`, optional) to specify which lineages to plot heatmaps for

In [ ]:
fig, axs = via.plot_gene_trend_heatmaps(via_object=v0, df_gene_exp=df_magic, cmap='plasma',
                             marker_lineages=[7,11])
fig.set_size_inches(5,5)

## Driver Genes
Identify driver genes of a cell fate/lineage.
#### Key Parameters
- `gene_exp` (`dataframe`) single-cell level gene expression of selected genes
- `lineage` (`integer`) to specify which lineages to compute driver genes
#### Optional Parameters
- `clusters` (`list`, optional) to manually specify cell clusters that belong in the lineage
- `conf_int` (`float`, optional) for computing

In [ ]:
df = adata.to_df()
driver_gene = via.compute_driver_genes(v0, gene_exp=df, lineage=24) # Here we select cell fate cluster 24

Filter gene lists for positively correlated driver genes with cutoff of correlection > 0.5.

In [ ]:
driver_upreg = driver_gene[driver_gene['corr']>0.5].sort_values('ci_low',ascending=False)
driver_upreg

Plot driver genes using `via.get_gene_expression()`.

In [ ]:
fig, axs=via.get_gene_expression(v0, gene_exp=df[driver_upreg.head(4).index])
fig.set_size_inches(14,3)

We can also plot driver genes directly from `via.get_gene_expression()`. Set parameters `driver_genes=True` and provide a terminal cluster to `driver_lineage`. This gives us top 3 upregulated and downregulated driver genes.

In [ ]:
fig, axs=via.get_gene_expression(v0, driver_genes=True, driver_lineage=24)
fig.set_size_inches(14,6)

## Various Visualizations
### Visualizations of the trajectory can be plotted at various edge resolutions

## Edge bundle plot
#### Key Parameters (default):
- `alpha_bundle_factor=1`
- `linewidth_bundle=2`
- `cmap:str = 'plasma_r'`
- `size_scatter:int=2`
- `alpha_scatter:float = 0.2`
- `headwidth_bundle=0.1`

## Animated Stream plot

In [ ]:
# sc.settings.set_figure_params(dpi=120, facecolor='white')
# #the streamlines load very slowly through the Notebook, so see the code below to open the file and view the animation properly
# via.animate_streamplot(v0, embedding=tsnem, cmap_stream='Blues', scatter_size=200, scatter_alpha=0.2, marker_edgewidth=0.15, 
#                         density_grid=0.7, linewidth=0.1, segment_length=1.5, saveto='./Figures/HumanCD34_animation.gif')

In [ ]:
from IPython.display import Image
with open('./Figures/HumanCD34_animation.gif','rb') as file:
    display(Image(file.read()))

## Animated edge bundle plot

In [ ]:
# via.animate_atlas(via_object=v0, extra_title_text='test animation', n_milestones=None,
#                         saveto='./Figures/human_edgebundle_test.gif')

In [ ]:
from IPython.display import Image
with open('./Figures/human_edgebundle_test.gif','rb') as file:
    display(Image(file.read()))